In [1]:
!pip install pymupdf

In [2]:
import requests

import requests
import fitz  # PyMuPDF
from io import BytesIO

import nltk
nltk.download('punkt_tab')
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from collections import Counter
import string

[nltk_data] Downloading package punkt_tab to /Users/sarah/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package punkt to /Users/sarah/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
fasting_metabolic_query = {} # TODO: name this whatever for your particular query, make sure you replace future references to smoking_query with the new name

def get_papers(query, limit=100):
    url = "https://api.semanticscholar.org/graph/v1/paper/search"
    params = {
        "query": query,
        "limit": limit,
        "fields": "title,authors,year,url,citationCount,referenceCount"
    }
    response = requests.get(url, params=params)
    return response.json()["data"]

papers = get_papers("Effects of fasting on metabolic health") # TODO: Change this for respective query
for i, paper in enumerate(papers, 1):
  if i <= 30 or i > 70:
    authors = paper.get('authors', 'N/A')
    if authors != 'N/A':
      authors = [author['name'] for author in authors]
    fasting_metabolic_query[paper["url"]] = {'title': paper.get("title", "N/A"), 'authors': authors, 'year': paper.get("year", 0),
                                   'citations': paper.get("citationCount", 0), 'references': paper.get("referenceCount", 0),
                                   'relevance': 2 if i <= 30 else 1}

In [4]:
for index, key in enumerate(fasting_metabolic_query.keys()):
    print(f"{index+1}: {key}") #first 30 are relevant documents (relevance = 2), last 30 are semi-relevant documents (relevance = 1)

1: https://www.semanticscholar.org/paper/8b7c778b54bc7863382b68b43651d3e65c3d3f06
2: https://www.semanticscholar.org/paper/7e32516ec04f6c6bad8f32bed5868301398622d1
3: https://www.semanticscholar.org/paper/7445100b833fdc2b273d2011b37c2e95f17b91fc
4: https://www.semanticscholar.org/paper/a1ba22627cbb33cb87855e217a8d06a5857d330f
5: https://www.semanticscholar.org/paper/6f20f5c3016a5419863035a1cbc108ad2f417995
6: https://www.semanticscholar.org/paper/4b79bd312c9fdd2e5b7631e2aa514a7dc75bb820
7: https://www.semanticscholar.org/paper/960e75207ba062e33b5e3d160f75981d7d26a434
8: https://www.semanticscholar.org/paper/4d6ef5bce12ddfa23910dd48a64e47846dfcab1d
9: https://www.semanticscholar.org/paper/beb682cd64e8019f501fc7b3210bbdd101be3da7
10: https://www.semanticscholar.org/paper/13c3702c266c5806e6ec208c1d6460d920e0445f
11: https://www.semanticscholar.org/paper/b8747af5907756a21a3d7816e371f7fbbad469e3
12: https://www.semanticscholar.org/paper/d22201e704a930419373e8839292965ce946defc
13: https://w

In [10]:
import os
def extract_text_from_pdf_url(pdf_urls):
    urls = list(fasting_metabolic_query.keys())

    for i, pdf_url in enumerate(pdf_urls):
            if pdf_url.startswith("file://"):
                # Handle local file
                local_path = pdf_url.replace("file://", "")
                if os.path.exists(local_path):
                    with fitz.open(local_path) as doc:
                        pdf_text = "\n".join([page.get_text() for page in doc])
                else:
                    print(f"File not found: {local_path}")

            else:
                response = requests.get(pdf_url)
                response.raise_for_status()
                if "application/pdf" in response.headers.get("Content-Type", ""):
                    with fitz.open(stream=BytesIO(response.content), filetype="pdf") as doc:
                        pdf_text = "\n".join([page.get_text() for page in doc])
                else:
                    print(f"Invalid PDF format or non-PDF file at {pdf_url}")

                    

            # Tokenize and clean
            tokens = word_tokenize(pdf_text)
            tokens = [token for token in tokens if token not in string.punctuation]
            word_counts = Counter(tokens)
            fasting_metabolic_query[urls[i]]['tokens'] = word_counts

In [11]:
# TODO
pdf_urls = [
    'https://pdfs.semanticscholar.org/988c/e762e97ec9c21be3f9da38001e58dc44ad52.pdf',
    'https://irep.ntu.ac.uk/id/eprint/43043/1/1444715_a1711_Clayton.pdf',
    'https://pdfs.semanticscholar.org/0881/0341a5bd26686d4cd7711895d7865d6e914d.pdf',
    'file:///Users/sarah/Downloads/aquaculture.pdf',
    'https://pdfs.semanticscholar.org/2168/33cb2f3c7c8706f508b77f2cce8fdac77f11.pdf',
    'https://pdfs.semanticscholar.org/7505/39e99db0aef7a83607b0db197ea7c7db0da0.pdf',
    'https://pdfs.semanticscholar.org/ace0/61936dadee232b37b71e46a24b602c8cc501.pdf',
    'https://www.anataramedicine.com/wp-content/uploads/2020/01/Effects-of-Intermittent-Fasting-on-Health.pdf',
    'file:///Users/sarah/Downloads/unravelling.pdf',
    'https://bmcbiol.biomedcentral.com/counter/pdf/10.1186/s12915-021-00987-5.pdf',

    'https://dmsjournal.biomedcentral.com/counter/pdf/10.1186/s13098-023-01190-y.pdf',
    'file:///Users/sarah/Downloads/nuad026.pdf',
    'https://www.scielo.cl/pdf/rmc/v151n1/0717-6163-rmc-151-01-0081.pdf',
    'file:///Users/sarah/Downloads/nuae0014.pdf',
    'https://pdfs.semanticscholar.org/3134/f8d998f0acc52b6d7c2ca48d6a98706afb5b.pdf',
    'file:///Users/sarah/Downloads/NEJMx190038.pdf',
    'https://pdfs.semanticscholar.org/0e81/f52f1437991a1225e8e8c18f46425af13370.pdf',
    'https://pdfs.semanticscholar.org/75b6/a9086721436e6d802b070f0d94b366079f95.pdf',
    'file:///Users/sarah/Downloads/000647143900001.pdf',
    'file:///Users/sarah/Downloads/nihms-1737120.pdf',

    'file:///Users/sarah/Downloads/fendo-08-00340.pdf',
    'file:///Users/sarah/Downloads/1.3.4.pdf',
    'file:///Users/sarah/Downloads/1-s2.0-S0022316622002796-main.pdf',
    'https://pdfs.semanticscholar.org/22c5/41d7c695acdd760bfd93f06d6fd88591706d.pdf',
    'file:///Users/sarah/Downloads/ctondelli,+27-akkocai-7588.pdf',
    'https://www.me-foreningen.no/wp-content/uploads/2019/12/Int-fasting-2017-annurev-nutr-071816-064634.pdf',
    'https://pdfs.semanticscholar.org/636b/c16b17dcd686b4805a6ffd7d988f193d0400.pdf',
    'file:///Users/sarah/Downloads/CMC-31-14-01.pdf',
    'https://pdfs.semanticscholar.org/3cb1/4f0e616d46dd217e446e0cfd3b1a9ab71fc3.pdf',
    'file:///Users/sarah/Downloads/fpubh-10-751126.pdf',

    'https://pdfs.semanticscholar.org/75f0/4ad827f9e454b36fd160305616f8312dcba5.pdf',
    'file:///Users/sarah/Downloads/SJG-30-53.pdf',
    'file:///Users/sarah/Downloads/Phytotherapysupplement.pdf',
    'file:///Users/sarah/Downloads/s41574-023-00817-4.pdf',
    'file:///Users/sarah/Downloads/nihms-1064487.pdf',
    'https://pdfs.semanticscholar.org/b1e5/8cdc26392459e1f74a9bc1280f883d7927f5.pdf',
    'file:///Users/sarah/Downloads/s11695-021-05724-z.pdf',
    'https://edoc.mdc-berlin.de/id/eprint/22548/1/22548oa.pdf',
    'file:///Users/sarah/Downloads/FASSB.pdf',
    'https://pdfs.semanticscholar.org/2e53/e66a28b01db1faf817c6923c64214fc11e33.pdf',

    'https://pdfs.semanticscholar.org/232f/98f2b70699ffa58dc7ec4704a8763170fced.pdf',
    'file:///Users/sarah/Downloads/physio.pdf',
    'https://pdfs.semanticscholar.org/124a/432ae589c5ece6fd3a1354d011ce1af0bd22.pdf',
    'file:///Users/sarah/Downloads/kjn-43-57.pdf',
    'file:///Users/sarah/Downloads/EMMM-14-e14418.pdf',
    'file:///Users/sarah/Downloads/nihms-1651371.pdf',
    'https://pdfs.semanticscholar.org/e81a/40fb771da5fd51417cf677367863fea01875.pdf',
    'https://pdfs.semanticscholar.org/04fb/96560974e2c609660d5653d5ec732a56e3d0.pdf',
    'https://www.kjcn.or.kr/upload/pdf/DJOSBK_2008_v13n1_46.pdf',
    'https://www.cambridge.org/core/services/aop-cambridge-core/content/view/1E4307FAFD57B566BE13193A09037673/S2048679019000338a.pdf/div-class-title-changes-in-human-gut-microbiota-composition-are-linked-to-the-energy-metabolic-switch-during-10-d-of-buchinger-fasting-div.pdf',

    'https://pdfs.semanticscholar.org/3740/7ab9f01205a43fa0fd2aabb14d71e35d2454.pdf',
    'https://pdfs.semanticscholar.org/0018/d791e57c08ceb7452a11974542886570b89c.pdf',
    'https://pdfs.semanticscholar.org/e33d/c07dc648a2d05403dc64ecd1a74fac97694d.pdf',
    'https://pdfs.semanticscholar.org/eac7/433af95ba561feafb3bb2d3c0cc71f8fd70e.pdf',
    'https://bmcsportsscimedrehabil.biomedcentral.com/counter/pdf/10.1186/s13102-019-0141-9.pdf',
    'https://pdfs.semanticscholar.org/e857/eb9204ce554a7607f4c369f7d57ddbca3e41.pdf',
    'file:///Users/sarah/Downloads/fmicb-10-01809.pdf',
    'file:///Users/sarah/Downloads/justesen-et-al-interactions-of-lipid-genetic-risk-scores-with-estimates-of-metabolic-health-in-a-danish-population.pdf',
    'file:///Users/sarah/Downloads/strengthtrain.pdf',
    'file:///Users/sarah/Downloads/1-s2.0-S1051227619303139.pdf'
    ] # fill out as list of pdf urls (need to get them manually), should be of size 60 (make sure it's in the same order as the urls in the previous box)
len(pdf_urls)

60

In [12]:
extract_text_from_pdf_url(pdf_urls)

In [8]:
print(fasting_metabolic_query[list(fasting_metabolic_query.keys())[0]]) # can use this to check specific url values

{'title': 'Effects of Different Types of Intermittent Fasting Interventions on Metabolic Health in Healthy Individuals (EDIF): A Randomised Trial with a Controlled-Run in Phase', 'authors': ['Daniel Herz', 'Sebastian Karl', 'Johannes W Weiß', 'Paul Zimmermann', 'Sandra Haupt', 'Rebecca T. Zimmer', 'Janis R. Schierbauer', 'Nadine B Wachsmuth', 'Maximilian P Erlmann', 'T. Niedrist', 'Kayvan Khoramipour', 'Thomas Voit', 'Sian Rilstone', 'Harald Sourij', 'O. Moser'], 'year': 2024, 'citations': 7, 'references': 45, 'relevance': 2, 'tokens': Counter({'the': 444, 'of': 218, 'in': 200, 'and': 196, '±': 154, 'fasting': 111, 'to': 110, 'p': 110, 'a': 102, 'for': 78, 'was': 78, '16/8': 72, 'were': 69, 'ADF': 67, '20/4': 65, 'significant': 58, 'The': 55, 'group': 53, 'with': 50, 'on': 48, 'as': 46, 'no': 46, 'participants': 45, 'study': 43, 'IF': 42, 'CrossRef': 41, 'cohort': 39, 'groups': 36, 'PubMed': 35, 'In': 33, 'by': 32, 'during': 32, '16': 30, 'phase': 30, 'h': 29, 'intervention': 29, '1': 

In [13]:
import pickle

# save query dict
with open("word_counts.pkl", "wb") as f:
    pickle.dump(fasting_metabolic_query, f)

# load query dict
with open("word_counts.pkl", "rb") as f:
    my_data = pickle.load(f)
